## Microsoft Fabric Monitoring Hub API Script

This Python script fetches and processes job history data from Microsoft Fabric's Monitoring Hub using an unofficial API endpoint. It leverages requests to make the API call and pandas to clean and analyze the data, providing insights into job durations, outcomes, and ownership.

Note: This uses an unofficial API discovered via browser developer tools. Use with caution, as Microsoft may change or disable this endpoint.

In [ ]:
import requests
import pandas as pd

# Authenticate and get token (replace 'pbi' with your actual credential source)
token = notebookutils.credentials.getToken('pbi')
headers = {"Authorization": f"Bearer {token}"}

# Define parameters for API request
params = {
    "limit": 5000,  # Number of records to fetch
    "startTime": "2025-04-24T00:00:00.000Z",  # Start date
    "endTime": "2025-04-28T23:59:28.484Z",  # End date
    "artifactTypes": "SynapseNotebook,Pipeline,Lakehouse",  # Types to monitor 
    "accept": "application/json",  # Accept JSON response
    "activityId": "15f7b7c7-21f2-48db-b164-e2801527e933",  # Example activity ID
    "accept-language": "en-US,en;q=0.9"  # Language preference
}

# Fetch data from Monitoring Hub API (URL redacted for security)
response = requests.get("https://wabi-xxxxx-xxxxx-x-xxxxx-xxxxx.analysis.windows.net/metadata/monitoringhub/histories", headers=headers, params=params)

# Raise an error if request failed
response.raise_for_status()

# Load and normalize JSON response
df = pd.json_normalize(response.json())

# Parse datetime columns
date_columns = ["jobStartTimeUtc", "jobEndTimeUtc", "jobScheduleTimeUtc", "jobNextCheckTimeUtc"]
df[date_columns] = df[date_columns].apply(pd.to_datetime)

# Map job outcome to readable format
df["jobOutcome"] = df["isSuccessful"].map({True: "Success", False: "Failed"})

# Categorize runs based on artifact type
df["runCategory"] = df["artifactJobType"].fillna("Unknown")

# Reorder columns for better readability
preferred_order = ["artifactJobInstanceId", "artifactName", "artifactType", "workspaceName", "ownerUser.userPrincipalName", "jobScheduleTimeUtc", "jobStartTimeUtc", "jobEndTimeUtc", "jobOutcome", "statusString"]

all_columns = df.columns.tolist()
remaining_columns = [col for col in all_columns if col not in preferred_order]
ordered_columns = preferred_order + remaining_columns
df = df[ordered_columns]

# Display sample rows
display(df)


##### Usage





Replace the placeholder API URL with the one you discover via browser developer tools (Inspect → Network tab).



Ensure you have a valid token for authentication (e.g., via notebookutils in a Fabric notebook environment).



Run the script in a Python environment with requests and pandas installed.

##### Parameters





limit: Number of records to fetch (e.g., 5000).



startTime/endTime: Time range for job history (ISO 8601 format).



artifactTypes: Comma-separated list of item types to monitor (e.g., SynapseNotebook, Pipeline, Lakehouse).

##### Output

The script outputs a DataFrame with columns like artifactJobInstanceId, artifactName, jobStartTimeUtc, jobOutcome, and more, providing a clear view of job activities in Microsoft Fabric.